In [50]:
import os
from pyspark import SparkContext
from operator import add

# Set parent directory

In [2]:
base = os.path.abspath(os.pardir)
data_dir = os.path.join(base, "work", "files")
data_dir

'/home/jovyan/work/files'

# Spark Context

In [3]:
sc = SparkContext(master="local", appName="trasnformacionesYAacciones")

21/08/20 18:03:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
sc

<SparkContext master=local appName=trasnformacionesYAacciones>

In [5]:
rdd1 = sc.parallelize([1,2,3])
type(rdd1)

pyspark.rdd.RDD

In [6]:
rdd1.collect() # Don't use...

[1, 2, 3]

In [7]:
equipos_olimpicos_rdd = sc.textFile(f"{data_dir}/paises.csv").map(lambda line: line.split(","))

In [8]:
equipos_olimpicos_rdd.take(15)

[['id', 'equipo', 'sigla'],
 ['1', '30. Februar', 'AUT'],
 ['2', 'A North American Team', 'MEX'],
 ['3', 'Acipactli', 'MEX'],
 ['4', 'Acturus', 'ARG'],
 ['5', 'Afghanistan', 'AFG'],
 ['6', 'Akatonbo', 'IRL'],
 ['7', 'Alain IV', 'SUI'],
 ['8', 'Albania', 'ALB'],
 ['9', 'Alcaid', 'POR'],
 ['10', 'Alcyon-6', 'FRA'],
 ['11', 'Alcyon-7', 'FRA'],
 ['12', 'Aldebaran', 'ITA'],
 ['13', 'Aldebaran II', 'ITA'],
 ['14', 'Aletta', 'IRL']]

In [9]:
equipos_olimpicos_rdd.map(lambda x: (x[2])).distinct().count()

231

In [10]:
(equipos_olimpicos_rdd
     .map(lambda x: (x[2], x[1]))
     .groupByKey()
     .mapValues(len)
     .take(5)
)

[('sigla', 1), ('AUT', 11), ('MEX', 9), ('ARG', 18), ('AFG', 1)]

In [11]:
(equipos_olimpicos_rdd
     .map(lambda x: (x[2], x[1]))
     .groupByKey()
     .mapValues(list)
     .take(2)
)

[('sigla', ['equipo']),
 ('AUT',
  ['30. Februar',
   'Austria',
   'Austria-1',
   'Austria-2',
   'Breslau',
   'Brigantia',
   'Donar III',
   'Evita VI',
   'May-Be 1960',
   '"R.-V. Germania; Leitmeritz"',
   'Surprise'])]

In [12]:
equipos_argentinos = equipos_olimpicos_rdd.filter(lambda l: "ARG" in l)
equipos_argentinos.collect()

[['4', 'Acturus', 'ARG'],
 ['37', 'Antares', 'ARG'],
 ['42', 'Arcturus', 'ARG'],
 ['43', 'Ardilla', 'ARG'],
 ['45', 'Argentina', 'ARG'],
 ['46', 'Argentina-1', 'ARG'],
 ['47', 'Argentina-2', 'ARG'],
 ['119', 'Blue Red', 'ARG'],
 ['238', 'Covunco III', 'ARG'],
 ['252', 'Cupidon III', 'ARG'],
 ['288', 'Djinn', 'ARG'],
 ['436', 'Gullvinge', 'ARG'],
 ['644', 'Matrero II', 'ARG'],
 ['672', 'Mizar', 'ARG'],
 ['774', 'Pampero', 'ARG'],
 ['843', 'Rampage', 'ARG'],
 ['1031', 'Tango', 'ARG'],
 ['1162', 'Wiking', 'ARG']]

- Cuando queremos contar todos los registros, usamos count
- Si queremos un aproximado, usaremos countApprox y le indicaremos durante cuantos milisegundos queremos que cuente. Si se termina el conteo antes de tiempo, no pasara nada pero si llegara al limite, regresara lo que logro contar hasta ese punto

In [13]:
equipos_olimpicos_rdd.count()

1185

In [14]:
equipos_olimpicos_rdd.countApprox(20)

1185

In [15]:
deportista_olimpico_rdd = sc.textFile(f"{data_dir}/deportista.csv").map(lambda l: l.split(","))
deportista_2_olimpico_rdd = sc.textFile(f"{data_dir}/deportista2.csv").map(lambda l: l.split(","))

# Union

In [16]:
deportista_olimpico_rdd = deportista_olimpico_rdd.union(deportista_2_olimpico_rdd)

In [17]:
deportista_olimpico_rdd.count()

135572

# Join

In [18]:
equipos_olimpicos_rdd.top(2)

[['id', 'equipo', 'sigla'], ['999', 'Stella-2', 'NOR']]

In [19]:
deportista_olimpico_rdd.top(2)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['99999', 'Alexander Grant Alick Rennie', '1', '32', '182', '71', '967']]

In [21]:
(deportista_olimpico_rdd
     .map(lambda l: (l[-1], l[:-1]))
     .join(equipos_olimpicos_rdd.map(lambda x: (x[0],x[2])))
     .takeSample(False, 6, 25) # Incluid duplicados, cantidad, semilla aleatoria
)

[('362', (['131505', 'Steven Woodburn', '1', '24', '185', '90'], 'FRA')),
 ('967', (['13626', 'Jill Brresen', '2', '22', '170', '57'], 'RSA')),
 ('482', (['44299', 'Gumundur Gumundsson', '1', '23', '174', '77'], 'ISL')),
 ('970', (['68062', 'Lee MinHui', '2', '28', '174', '65'], 'KOR')),
 ('794', (['92442', 'Luis Paz Zoldan', '1', '19', '187', '82'], 'PER')),
 ('413', (['26822', 'Jared Mark Deacon', '1', '24', '185', '77'], 'GBR'))]

In [24]:
resultado = sc.textFile(f"{data_dir}/resultados.csv").map(lambda l: l.split(","))

In [25]:
resultado_ganador = resultado.filter(lambda l: "NA" not in l[1])

In [27]:
resultado_ganador.take(3)#.takeSample(False, 5)

[['resultado_id', 'medalla', 'deportista_id', 'juego_id', 'evento_id'],
 ['4', 'Gold', '4', '2', '4'],
 ['38', 'Bronze', '15', '7', '19']]

In [36]:
paises_medallas = (deportista_olimpico_rdd
     .map(lambda l: (l[-1], l[:-1]))
     .join(equipos_olimpicos_rdd.map(lambda x: (x[0],x[2])))
     .join(resultado_ganador)
)

In [37]:
paises_medallas.take(6)

[('74',
  ((['65', 'Patimat Abakarova', '2', '21', '165', '49'], 'AZE'), 'Gold')),
 ('74', ((['129', 'Ruslan Abbasov', '1', '22', '181', '74'], 'AZE'), 'Gold')),
 ('74', ((['130', 'Tural Abbasov', '1', '18', '182', '76'], 'AZE'), 'Gold')),
 ('74', ((['131', 'Tran Abbasova', '2', '33', '159', '53'], 'AZE'), 'Gold')),
 ('74',
  ((['335', 'Abdulqdir Abdullayev', '1', '28', '188', '91'], 'AZE'), 'Gold')),
 ('74',
  ((['336', 'Arif Yadulla Abdullayev', '1', '27', '164', '60'], 'AZE'),
   'Gold'))]

In [45]:
valores_medallas = {"Gold": 7, "Silver": 5, "Bronze": 4}

In [46]:
paises_medallas.map(lambda x: (x[1][0][-1], x[1][1])).take(4)

[('AZE', 'Gold'), ('AZE', 'Gold'), ('AZE', 'Gold'), ('AZE', 'Gold')]

In [51]:
paises_medallas = paises_medallas.map(lambda x: (x[1][0][-1], valores_medallas[x[1][1]]))#.distinct().takeSample(False,4)

In [54]:
conclusion = paises_medallas.reduceByKey((add)).sortBy(lambda x: x[1], ascending=False)

In [56]:
conclusion.take(10)

[('CAN', 32538),
 ('ARG', 12520),
 ('HUN', 10860),
 ('MEX', 6124),
 ('RSA', 3788),
 ('BLR', 3580),
 ('LTU', 1535),
 ('MGL', 1460),
 ('USA', 1342),
 ('AZE', 1218)]

In [57]:
sc.stop()